### import 및 기본변수

In [47]:
import pandas as pd
import numpy as np
import re, os, functions
from os.path import join
import warnings
warnings.simplefilter("ignore")

debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str, '관리자비고':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str, '신고계좌':str, '입금계좌':str}
credit_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}
reduction_dtype = {'채무자키':str, '계좌키':str, '감면키':str}
installment_dtype = {'채무자키':str, '계좌키':str, '분납키':str}

##################################
company = "솔림"      # 솔림 or 대성
basedate = "250423" 
wd = join(r"D:\3.자산\전산 dataset", company)
##################################
ext = ".xlsx"  
read_dir = join(wd, basedate)
write_dir = join(wd, basedate, "pkl")

# pkl 폴더 만들기
if not os.path.exists(write_dir) :
    os.mkdir(write_dir)

#### 개인회생

In [35]:
# 엑셀 읽기, 정렬(사건-컷오프-채권번호), pkl 저장
filename = functions.키워드로파일명찾기(read_dir, "개인회생새창", "기준데이터", 전체경로=False).split('.')[0]
raw_data_ori = pd.read_excel(join(read_dir, filename+".xlsx"), dtype=rehabilitation_dtype).fillna("")

# 진행내용 표시 했는지 체크
if raw_data_ori.loc[0,"진행내용"] == "" :
        print("개인회생 엑셀다운로드시 우클릭후 '1.채권자목록,일반내용,진행내용 표시'를 선택해야 합니다.")

# pkl 저장
raw_data_ori.to_pickle(join(write_dir, filename+".pkl"))

#### 신용회복 계좌별진행상황

In [46]:
# 계좌별 진행상황 파일 읽기
########################################################
약식병합 = True ############# 금요일(일요일)이 컷오프가 아니라면 약식병합해야-------
basedate = basedate # "250423" # 정식 업로드결과 파일 날짜 컷오프일에 맞춰 수정
company = company  # 솔림 or 대성
########################################################

nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"

# pkl 폴더 없으면 만들기
write_path_base = join(path_base, "pkl")
if not os.path.exists(write_path_base) :
    os.mkdir(write_path_base)

# 읽을 파일명
suffix = "통합" if company!="대성" else "대성"

filename = basedate + "_계좌별 진행상황 조회_" + suffix

filename_정식 = filename + "_업로드결과"
filename_약식 = filename + "_약식_업로드결과"

# 계좌별 진행상황 조회 파일 읽기 및 pkl 저장
    # 정식
정식_ori = pd.read_excel(join(path_base, filename_정식+".xlsx" ), dtype=nauri_dtype).fillna("")
정식_ori.to_pickle(join(write_path_base, filename_정식+".pkl"))
    # 약식
if 약식병합 : 
    약식_ori = pd.read_excel(join(path_base, filename_약식+".xlsx"), dtype=nauri_dtype).fillna("")
    약식_ori.to_pickle(join(write_path_base, filename_약식+".pkl"))
    

#### 전산 raw_data

In [48]:
#########################################
세부주소 = False # 상세주소 제외할지 모두 포함할지
# 초본 > 자택 > 직장 > 기타 우선순위 적용은 항상 한다.
#########################################
파일검색어 = { # [타입, 제외키워드]
    # "채무자조회" : [debt_dtype,""],
    # "계좌조회" : [account_dtype,""],
    # "보증인새창" : [grt_dtype,"새보증인상태"],
    "법조치조회" : [event_dtype,""],
    # "감면조회" : [reduction_dtype,""],
    # "약정분납새창" : [installment_dtype,"기준데이터"],
    # "입금조회.*당월" : [deposit_dtype,""]
}
    
for i, v in 파일검색어.items() : 
    print(i, '파일변환시작')
    filename = os.path.splitext(functions.키워드로파일명찾기(read_dir, i, v[1],전체경로=False))[0]
    df = pd.read_excel(join(wd, basedate, filename+ext), dtype = v[0]).fillna("")
    
    # 채무자 관리자비고(새채무자키) 검사
    if re.search("채무자", i) :
        if len(df.query("관리자비고==''")) > 1 :
            print("새채무자키 부여 필요")
            print(len(nothing)) # 오류를 위한 코드
        
        # 예수금제외
        df.drop(df[(df.성명=="예수금")].index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        # 채무자 주소 보정  
        cols = ["초본주소", "초본번지인", "초본우편번호", "자택주소","자택번지인","자택우편번호","직장주소","직장번지인", "직장우편번호","기타주소", "기타번지인", "기타우편번호"]
        df[cols] = df[cols].applymap(lambda x: x.strip() if isinstance(x, str) else x)


        # 주소채우기 : 초본이 없는 경우 자택 > 직장 > 기타 주소를 초본칸에 넣기
        mask = (df.초본주소=="")
        df.loc[mask,["초본주소", "초본번지인", "초본우편번호"]] = df.loc[mask, ["자택주소", "자택번지인", "자택우편번호"]]
        mask = (df.초본주소=="")
        df.loc[mask,["초본주소", "초본번지인", "초본우편번호"]] = df.loc[mask, ["직장주소", "직장번지인", "직장우편번호"]]
        mask = (df.초본주소=="")
        df.loc[mask,["초본주소", "초본번지인", "초본우편번호"]] = df.loc[mask, ["기타주소", "기타번지인", "기타우편번호"]]
    
    # 계좌 예수금 제거
    if re.search("계좌", i) :
        df.drop(df[(df.채무자명=="예수금")].index, inplace=True)
        df.reset_index(drop=True, inplace=True)

    # 법조치 법원사건번호 없는 것 제거
    if re.search('법조치', i) :
        index =  df[(df.관할법원=="0") | (df.관할법원=="") | (df.사건번호=="0") | (df.사건번호=="")].index
        df.drop(index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        # 사건번호 역순정렬
        df.sort_values('사건번호', ascending=False, inplace=True)
    
    # pkl 저장
    df.to_pickle(join(write_dir, filename+".pkl"))

법조치조회 파일변환시작
